In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
from scipy.stats import entropy  # For KL divergence
import matplotlib.pyplot as plt

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

def get_hidden_state_probs(sentence):
    """Get the probability distribution of hidden states for a sentence."""
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Extract last hidden state (shape: [1, seq_len, hidden_dim])
    last_hidden_state = outputs.hidden_states[-1].squeeze(0)

    # Normalize into probability distribution (softmax over hidden states per token)
    probs = F.softmax(last_hidden_state, dim=-1).numpy()
    
    return probs  # Shape: [seq_len, hidden_dim]

def compute_kl_divergence(sentence_before, sentence_after):
    """Compute KL divergence between hidden state distributions before and after completion."""
    probs_before = get_hidden_state_probs(sentence_before)
    probs_after = get_hidden_state_probs(sentence_after)

    # Ensure both distributions have the same number of tokens
    min_len = min(probs_before.shape[0], probs_after.shape[0])
    probs_before = probs_before[:min_len]
    probs_after = probs_after[:min_len]

    # Compute KL divergence for each token and take the mean
    kl_divs = [entropy(probs_before[i], probs_after[i]) for i in range(min_len)]
    avg_kl_div = sum(kl_divs) / len(kl_divs)

    return avg_kl_div

def calculate_bayesian_surprise(text, target_window_size=100, context_window_size=5):
    """Calculate Bayesian surprise (KL divergence) at each position in the text using a moving window."""
    tokens = tokenizer.tokenize(text)
    surprises = []
    start_words = []

    for i in range(len(tokens) - target_window_size - context_window_size):
        # Define context window
        context = tokens[i : i + target_window_size]  # Take current sliding window
        context_str = tokenizer.convert_tokens_to_string(context)

        if i + target_window_size < len(tokens):
            new_context = tokens[i : i + target_window_size + context_window_size]  # Extend window
            new_context_str = tokenizer.convert_tokens_to_string(new_context)

            # Compute Bayesian surprise (KL divergence)
            kl_div = compute_kl_divergence(context_str, new_context_str)
            surprises.append(kl_div)
            start_words.append(tokenizer.convert_tokens_to_string([tokens[i+target_window_size+context_window_size]]))  # Track starting word

    return start_words, surprises

# Example Text
text_1 = (
    "user: # The Hobbit: A Tale of Unexpected Courage\n\n"
    "In a hole in the ground there lived a hobbit. Not a dirty, nasty hole, but a cozy hobbit-hole named Bag End, where Bilbo Baggins lived a quiet, respectable life until the wizard Gandalf arrived at his round green door. Though Bilbo prided himself on his predictable routines and comfortable lifestyle, destiny had other plans.\n\n"
    "That evening, thirteen dwarves led by the noble Thorin Oakenshield crowded into his dining room. They spoke of a lost kingdom under the Lonely Mountain, of treasures stolen by the dragon Smaug, and of their desperate quest to reclaim their homeland. Despite all his proper hobbit instincts telling him to stay home, something deep inside Bilbo – a tiny spark of his mother's adventurous Took blood – stirred at their songs of ancient gold and glory.\n\n"
    "Before he knew quite how it happened, Bilbo found himself running down the road without even a pocket handkerchief, joining this band of dwarves as their designated burglar. Through treacherous mountains and dark forests they traveled, facing dangers that Bilbo had never imagined in his comfortable Shire. When trolls captured his companions, it was the small hobbit's quick wit that saved them. Lost in goblin tunnels, he found not only a mysterious ring but also the courage to rely on himself.\n\n"
    "In the shadows of Mirkwood, while giant spiders threatened his friends, Bilbo discovered a bravery he never knew he possessed. No longer was he the same fussy hobbit who had left Bag End – he had grown into someone who would risk everything to help his companions, even when they doubted him.\n\n"
    "Finally reaching the Lonely Mountain, Bilbo faced the terrible dragon Smaug alone. Rather than raw strength, it was his cleverness and compassion that proved most valuable. When he found the Arkenstone – the heart of the mountain that Thorin desperately sought – he made the difficult choice to use it to prevent a war, even though he knew it might cost him his friendship with Thorin.\n\n"
    "In the great battle that followed, Bilbo witnessed the terrible cost of greed as Thorin finally overcame his dragon-sickness, but too late to prevent tragedy. Yet even in darkness, the hobbit's fundamental decency and loyalty never wavered. He stayed by Thorin's side until the end, learning that there are indeed many kinds of courage, and that the smallest person can change the course of the future.\n\n"
    "When at last Bilbo returned to Bag End, he was no longer quite respectable by hobbit standards. But he had gained something far more valuable – wisdom, courage, and stories worth telling. His modest hobbit hole was the same, but he was not, having learned that true wealth lies not in what you keep safely locked away, but in what you're willing to risk for friendship and what's right.\n\n"
    "The simple hobbit who had never ventured beyond his garden had become a legend, proving that greatness can come from the most unexpected places, and that the most important journeys change us in ways we never imagined.\n\n"
    "\nPlease ponder the question the user asks together with him.\n--------------\n"
    "user: I heard there is a new adventure in town. Do you think we should go on it?\n--------------\n"
    "assistant: Embarking on a new adventure can be exciting and transformative, much like Bilbo's journey in \"The Hobbit: A Tale of Unexpected Courage.\" Let's ponder a few aspects together:\n\n"
    "1. **Desire for Change:** Like Bilbo, is there a part of you intrigued by the unknown or seeking change from your daily routine? Sometimes curiosity can lead to great personal growth.\n\n"
    "2. **Potential Challenges:** Adventures often come with unexpected challenges. Are you prepared to face difficulties as they arise and handle them with resilience?\n\n"
    "3. **Personal Growth:** Consider how this adventure might help you learn more about yourself, develop new skills, or gain fresh perspectives.\n\n"
    "4. **Impact on Others:** Think about whether there are others who might benefit from your decision to embark on this journey—much like how Bilbo's choice impacted his friends and the broader world.\n\n"
    "5. **Safety and Comfort Zones:** It's important to ensure that the adventure aligns with your comfort level regarding safety and well-being but also encourages stepping slightly out of comfort zones for meaningful experiences.\n\n"
    "6. **Purpose and Goals:** Reflect on what you hope to achieve from this adventure—new friendships, stories worth sharing, or perhaps just a break from routine—and whether it aligns with your values.\n\n"
    "Ultimately, going on an adventure is a personal decision that involves weighing both the excitement of discovery against any potential risks or sacrifices involved. Whether you're inspired by Bilbo's courage or simply curious about what's out there, trust in your instincts while staying true to what feels right for you personally at this time!\n--------------\n"
    "user: Embarking on a new adventure indeed presents an invigorating prospect, much like Bilbo's transformative journey. Let's explore these aspects more deeply:\n\n"
    "1. **Desire for Change:** If you feel a pull towards the unknown or find yourself seeking a break from routine, that's worth listening to. Curiosity often sparks the beginning of a journey that can lead to unforeseen personal growth and satisfaction.\n\n"
    "2. **Potential Challenges:** No adventure is without its hurdles. Consider whether you're ready to face such challenges head-on and adapt as needed. Remember, resilience can be your greatest ally during uncertain times.\n\n"
    "3. **Personal Growth:** An adventure can teach you much about yourself, offering opportunities to expand your skills and perspectives in ways everyday life might not allow.\n\n"
    "4. **Impact on Others:** Your decision might resonate beyond your own experiences—like how Bilbo's journey affected those around him—possibly inspiring others or bringing about positive change within your community.\n\n"
    "5. **Safety and Comfort Zones:** Balancing safety with stepping out of comfort zones is crucial for meaningful experiences while ensuring well-being isn't compromised.\n\n"
    "6. **Purpose and Goals:** Clarify what you wish to achieve through this adventure, whether it's building new relationships, creating memorable stories, or simply taking time away from the mundane routine—and ensure these aims align with personal values.\n\n"
    "Ultimately, deciding to embark on an adventure is deeply personal and involves weighing excitement against potential risks or sacrifices involved. Whether inspired by tales like Bilbo's courage-filled expedition or driven by sheer curiosity about what lies beyond familiar horizons, trust yourself while staying true to what feels right at this moment!\n--------------\n"
)
    
text_2 = (
    "Lena had always been close to her grandmother, who lived in a cozy cottage at "
    "the edge of the forest. One evening, she received a letter from her grandmother "
    "asking her to visit urgently. It had been weeks since their last conversation, "
    "and Lena was curious about what could be so important. She packed a bag and "
    "made her way through the dense woods, where shadows seemed to play tricks on "
    "her mind. As she approached the cottage, the door was slightly ajar.\n\n"
    
    "Inside, everything appeared as usual, with the scent of lavender filling the "
    "air. \"Grandma?\" Lena called, but there was no answer. She ventured further "
    "in, and to her surprise, the living room was empty. A single note was placed "
    "on the table, reading: *\"Look behind the painting.\"*\n\n"
    
    "Lena's heart skipped. She had always been told not to meddle with the old "
    "paintings in the house. But curiosity got the best of her, and she gently "
    "moved the painting. To her shock, there was a hidden door behind it. Hesitant "
    "but determined, Lena opened the door and found a staircase leading down into "
    "darkness.\n\n"
    
    "As she descended, she felt an icy chill in the air. The basement was far "
    "larger than she expected, and it was filled with old trunks and mysterious "
    "relics. Suddenly, she heard a creak from behind. Turning, she was startled to "
    "see her grandmother standing there, grinning strangely. \"Grandma, what's "
    "going on?\" Lena asked, trembling.\n\n"
    
    "The woman chuckled softly. \"I suppose you're wondering about the truth.\" "
    "Lena was confused. \"What truth?\"\n\n"
    
    "Her grandmother stepped closer. \"I'm not your grandmother,\" she said coldly. "
    "Lena gasped. The woman in front of her wasn't her grandmother at all. But "
    "before Lena could react, the woman reached into her pocket and pulled out a "
    "small locket. She opened it, revealing a picture of Lena's real grandmother, "
    "smiling beside a young girl.\n\n"
    
    "With a shock, Lena realized the girl was herself—*as a child*. But that "
    "wasn't possible. She hadn't been here in years, and that was when she "
    "understood. The woman in front of her wasn't just a stranger; she was her "
    "long-lost twin sister."
)

# Compute Bayesian surprise
start_words, bayesian_surprises = calculate_bayesian_surprise(text_1, target_window_size=100, context_window_size=10)

# Plot results
plt.figure(figsize=(100, 5))
plt.plot(range(len(bayesian_surprises)), bayesian_surprises, marker='o', linestyle='-', color='b')
plt.xticks(range(len(start_words)), start_words, rotation=90)
plt.axvline(x=0, color='gray', alpha=0.3, linewidth=1.5)  # Add vertical line at x=0
for x in range(1, len(start_words)):
    plt.axvline(x=x, color='gray', alpha=0.3, linewidth=1.5)
plt.xlabel("Starting Word of Context Window")
plt.ylabel("Bayesian Surprise (KL Divergence)")
plt.title("Bayesian Surprise Across Text Positions")
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


In [ ]:
# Plot results
plt.figure(figsize=(200, 5))
plt.plot(range(len(bayesian_surprises)), bayesian_surprises, marker='o', linestyle='-', color='b')
plt.xticks(range(len(start_words)), start_words, rotation=90)
plt.axvline(x=0, color='gray', alpha=0.3, linewidth=1.5)  # Add vertical line at x=0
for x in range(1, len(start_words)):
    plt.axvline(x=x, color='gray', alpha=0.3, linewidth=1.5)
plt.xlabel("Starting Word of Context Window")
plt.ylabel("Bayesian Surprise (KL Divergence)")
plt.title("Bayesian Surprise Across Text Positions")
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

def get_sentence_embedding(sentence):
    """Get the BERT embedding for the CLS token of a sentence."""
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the [CLS] token embedding (batch_size=1, seq_len, hidden_dim)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Shape: (1, hidden_dim)
    
    return cls_embedding.squeeze(0).numpy()  # Convert to NumPy for distance calculation

def compute_embedding_shift(sentence_before, sentence_after):
    """Compute cosine similarity between sentence embeddings before and after completion."""
    emb_before = get_sentence_embedding(sentence_before)
    emb_after = get_sentence_embedding(sentence_after)

    similarity = 1 - cosine(emb_before, emb_after)  # Cosine similarity (higher = more similar)
    
    return similarity

# Example sentences
sentence_before = "I pack my bathing suit and go to the"
sentence_after_1 = "I pack my bathing suit and go to the swimming pool."
sentence_after_2 = "I pack my bathing suit and go to the courtroom to defend myself."

# Compute semantic similarity scores
similarity_1 = compute_embedding_shift(sentence_before, sentence_after_1)
similarity_2 = compute_embedding_shift(sentence_before, sentence_after_2)

print(f"Semantic Similarity (swimming pool completion): {similarity_1:.4f} (high similarity)")
print(f"Semantic Similarity (courtroom completion): {similarity_2:.4f} (low similarity)")


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased", output_attentions=True)

def get_attention(sentence, max_len):
    """Compute mean attention across layers & heads with padding to match max sequence length."""
    inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=max_len)
    
    with torch.no_grad():
        outputs = model(**inputs)

    attentions = torch.stack(outputs.attentions)  # Shape: (num_layers, num_heads, seq_len, seq_len)
    
    return attentions.mean(dim=(0, 1))  # Average over layers & heads → Shape: (seq_len, seq_len)

# Example sentences
before = "I pack my bathing suit and go to the"
after1 = "I pack my bathing suit and go to the swimming pool."
after2 = "I pack my bathing suit and go to the courtroom to defend myself."

# Determine max sequence length for padding
max_seq_len = max(len(tokenizer.tokenize(before)), 
                  len(tokenizer.tokenize(after1)), 
                  len(tokenizer.tokenize(after2))) + 2  # +2 for [CLS] and [SEP] tokens

# Compute attention matrices with padding
attn_before = get_attention(before, max_seq_len)
attn_after1 = get_attention(after1, max_seq_len)
attn_after2 = get_attention(after2, max_seq_len)

# Compare attention reallocation
shift1 = (attn_after1 - attn_before).abs().sum().item()  # Low impact
shift2 = (attn_after2 - attn_before).abs().sum().item()  # High impact

print(f"Attention shift (swimming pool): {shift1:.4f} (low impact novelty)")
print(f"Attention shift (courtroom): {shift2:.4f} (high impact novelty)")

